# Lab 6: Building AWS Pipelines with Agentic Document Extraction

In this lab, you will build a pipeline that combines automated document processing with a conversational chatbot featuring memory and visual grounding.

**Learning Objectives:**
- Build an event-driven, serverless  pipeline on AWS
- Deploy a Lambda function that parses PDFs automatically using LandingAI ADE
- Ingest parsed documents into Amazon Bedrock Knowledge Base
- Create a Strands Agent with persistent memory and visual grounding

## Background

In the previous lab, you built a RAG pipeline using local components. This lab migrates to the cloud on AWS:

| Lab 5 | Lab 6 | Difference |
|-----------------|----------------|----------|
| Local files | **Amazon S3** | Scalable storage |
| Local scripts | **AWS Lambda** | Serverless compute |
| ChromaDB | **Bedrock Knowledge Base** | Managed vector database |
| OpenAI | **Amazon Titan** | AWS embedding model |
| LangChain | **Strands Agents** | AWS agent framework |


## Outline

**Part 1: Setting Up the Lambda Function**
- [Step 1: Environment Setup](#step1)
- [Step 2: Initialize AWS Clients](#step2)
- [Step 3: Create the Deployment Package](#step3)
- [Step 4: Create the IAM Role](#step4)
- [Step 5: Deploy the Lambda Function](#step5)
- [Step 6: Set Up the S3 Trigger](#step6)

**Part 2: Building the Knowledge Base**
- [Step 7: Upload Documents for Processing](#step7)
- [Step 8: Connect to the Bedrock Knowledge Base](#step8)
- [Step 9: Ingest Documents into the Knowledge Base](#step9)

**Part 3: Building the Agent**
- [Step 10: Create the Search Tool with Visual Grounding](#step10)
- [Step 11: Create Memory for the Agent](#step11)
- [Step 12: Create the Strands Agent](#step12)
- [Step 13: Interactive Chat](#step13)

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Prerequisites)</code>:</b> This lab assumes you have an AWS account with an S3 bucket (with input/output folders) and a Bedrock Knowledge Base connected to the output folder. Links for setting up these resources are provided in the <code>README</code> file.</p>

## Architecture Overview

The complete data flow:

1. **Upload**: User uploads PDF to S3 `input/` folder
2. **Trigger**: S3 automatically triggers Lambda function
3. **Parse**: Lambda uses LandingAI ADE to parse PDF into structured markdown
4. **Store**: Parsed output (markdown, grounding data, chunks) saved to S3 `output/` folder
5. **Index**: Bedrock Knowledge Base indexes documents for semantic search
6. **Query**: Users ask questions to the Strands agent with memory

<div align="center">
    <img src="images/architecture_1.png" width="700">
</div>

## Installing Required Packages

Install the AWS and agent packages:
- **boto3**: AWS SDK for Python
- **bedrock-agentcore**: Memory management for agents
- **strands-agents**: AWS-native agent framework

In [ ]:
# Install required packages
!pip install --quiet boto3 python-dotenv Pillow PyMuPDF bedrock-agentcore strands-agents 

<a id="step1"></a>

## Step 1: Environment Setup

Load environment variables containing AWS credentials and configuration from a `.env` file.

In [ ]:
import boto3, os, json
from dotenv import load_dotenv

# Load environment variables
_ = load_dotenv()

**Example `.env` file:**
```bash
# AWS Credentials
AWS_ACCESS_KEY_ID=your_aws_access_key
AWS_SECRET_ACCESS_KEY=your_aws_secret_key
AWS_REGION=your_aws_region

# S3 Bucket
S3_BUCKET=your_bucket_name

# Bedrock Configuration
BEDROCK_MODEL_ID=your_llm_id
BEDROCK_KB_ID=your_bedrock_knowledge_base_id
DATA_SOURCE_ID=your_data_source_id

# LandingAI ADE API Key
VISION_AGENT_API_KEY=your_vision_api_key
```

<a id="step2"></a>

## Step 2: Initialize AWS Clients

The `boto3` library connects to AWS services through **clients**. Each client provides access to a specific service:

| Client | Service | Purpose |
|--------|---------|--------|
| `s3_client` | Amazon S3 | Upload/download files, manage buckets |
| `lambda_client` | AWS Lambda | Deploy functions, configure triggers |
| `iam` | IAM | Create roles with permissions |
| `logs` | CloudWatch | Monitor Lambda execution |
| `bedrock_agent_runtime` | Bedrock | Query knowledge bases |
| `bedrock_runtime` | Bedrock | Call Claude models directly |

In [ ]:
session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION"),
)

# Create clients
s3_client = session.client("s3")
lambda_client = session.client("lambda")
iam = session.client("iam")  # Add IAM client for Lambda role management
logs = session.client("logs")  # CloudWatch Logs client for monitoring
bedrock_agent_runtime = session.client("bedrock-agent-runtime")
bedrock_runtime = session.client("bedrock-runtime")

# Building a Medical Chatbot with Memory

The pipeline is built in three parts:

### Part 1: Setting Up the Lambda Function (Steps 3-5)
Package code, create IAM role, deploy function.

<div align="center">
    <img src="images/steps3-5.png" width="700">
</div>

### Part 2: Setting Up the Trigger (Step 6)
Configure S3 to invoke Lambda on file uploads.

<div align="center">
    <img src="images/step6.png" width="200">
</div>

### Part 3: Building the Agent (Steps 7-12)
Upload documents, ingest into knowledge base, create agent with memory.

<div align="center">
    <img src="images/steps7-12.png" width="700">
</div>

### Loading Helper Functions

Helper functions in `lambda_helpers.py` handle AWS operations like creating deployment packages, configuring IAM roles, and setting up triggers.

In [ ]:
# Import helper functions
import pandas as pd
from lambda_helpers import *

print("Helper functions loaded")

<a id="step3"></a>

## Step 3: Create the Deployment Package

### What is a Lambda Deployment Package?

AWS Lambda requires your code and dependencies bundled in a **zip file**:

```
ade_lambda.zip
├── ade_s3_handler.py          ← Your code (handler function)
├── landingai_ade/             ← LandingAI ADE package
│   ├── __init__.py
│   └── ...
├── typing_extensions/         ← typing-extensions package
│   └── ...
└── boto3/                     ← AWS SDK (often pre-installed)
    └── ...
```

The helper function creates this package by:
1. Creating a temporary directory
2. Installing pip packages into that directory
3. Copying your source code files
4. Zipping everything together

In [ ]:
source_files = ["ade_s3_handler.py"]
requirements = ["landingai-ade", "typing-extensions"]

zip_path = create_deployment_package(
    source_files=source_files,
    requirements=requirements,
    output_zip="ade_lambda.zip",
    package_dir="ade_package"
)

###  `ade_s3_handler.py` File

This code runs inside Lambda when triggered:

1. **Event received**: S3 upload triggers Lambda with file information
2. **Validation**: Check it's a PDF, not a folder, and output doesn't exist
3. **Download**: PDF downloaded to Lambda's `/tmp` directory
4. **Parse**: ADE API parses PDF into markdown and chunks
5. **Upload**: Results saved to S3 output folder in three formats:
   - **Markdown**: Complete document in readable format
   - **Grounding JSON**: All chunks with bounding box coordinates
   - **Individual chunks**: One file per chunk for knowledge base indexing

<div align="center">
    <img src="images/flow_ade_handler.png" width="600">
</div>

### Understanding the Output Files

When a PDF is processed, Lambda produces three types of outputs in the S3 `output/` folder:

<div align="center">
    <img src="images/files.png" width="800">
</div>

<a id="step4"></a>


## Step 4: Create the IAM Role

### What is an IAM Role?

Lambda functions run in **isolated containers** with no inherent permissions. An **IAM role** grants the function permission to access specific AWS services.

| Permission | Purpose |
|------------|--------|
| `s3:GetObject` | Read PDFs from input folder |
| `s3:PutObject` | Write markdown to output folder |
| `s3:HeadObject` | Check if output already exists |
| `logs:CreateLogGroup` | Create CloudWatch log group |
| `logs:CreateLogStream` | Create log stream per execution |
| `logs:PutLogEvents` | Write log entries for debugging |

In [ ]:
role_arn = create_or_update_lambda_role(
    iam_client=iam,
    role_name="lambda-ade-exec-role",
    description="Execution role for LandingAI ADE Lambda"
)

<a id="step5"></a>


## Step 5: Deploy the Lambda Function

Deploy with both required components:
- **Deployment package**: Code (zip file)
- **IAM role**: Permissions

The configuration includes:
- **Environment variables**: Configuration accessible at runtime
- **Timeout**: 900 seconds (15 minutes) for larger PDFs
- **Memory**: 1024 MB RAM

In [ ]:
env_vars = {
    "VISION_AGENT_API_KEY": os.getenv("VISION_AGENT_API_KEY"),
    "ADE_MODEL": "dpt-2-latest",
    "INPUT_FOLDER": "input/",
    "OUTPUT_FOLDER": "output/",
    "S3_BUCKET": os.getenv("S3_BUCKET"),
    "FORCE_REPROCESS": "false"  # Set to "true" to reprocess all files even if outputs exist
}

response = deploy_lambda_function(
    lambda_client=lambda_client,
    function_name="ade-s3-handler",
    zip_file="ade_lambda.zip",
    role_arn=role_arn,
    handler="ade_s3_handler.ade_handler",
    env_vars=env_vars,
    runtime="python3.10",
    timeout=900,
    memory_size=1024
)

<a id="step6"></a>


## Step 6: Set Up the S3 Trigger

The Lambda function is deployed but won't run automatically yet. Configure S3 to **trigger Lambda when files are uploaded**.

S3 sends events when objects are created, modified, or deleted. We'll configure it to invoke our function when files are uploaded to the `input/` folder.

In [ ]:
# Trigger on all files in input/ folder
setup_s3_trigger(
    s3_client=s3_client,
    lambda_client=lambda_client,
    bucket=os.getenv("S3_BUCKET"),
    prefix="input/",
    function_name="ade-s3-handler",
    suffix=None  # Optional: set to ".pdf" to only trigger on PDF files
)

<a id="step7"></a>


## Step 7: Upload Documents for Processing

Upload medical PDF documents and watch the pipeline in action.

When you upload files to `input/`, the event-driven architecture automatically:
1. Detects new files
2. Triggers Lambda
3. Parses with ADE
4. Saves outputs to `output/`

<div align="center">
    <img src="images/folders_s3.png" width="750">
</div>

In [ ]:
# Upload medical documents to S3 input folder
local_folder = "medical/"

# Check if folder exists and upload
if os.path.exists(local_folder):
    count = upload_folder_to_s3(
        s3_client=s3_client,
        local_folder=local_folder,
        s3_prefix=f"input/{local_folder}",
        bucket=os.getenv("S3_BUCKET"),
        file_extensions=[".pdf", ".PDF"]
    )
    print(f"\n Waiting for automatic parsing to complete...")
    print("   (The Lambda function will automatically convert PDFs to markdown)")
else:
    print(f" Folder not found: {local_folder}")

### Monitoring Lambda Processing

Monitor processing progress in real-time via CloudWatch logs:

In [ ]:
stats = monitor_lambda_processing(
    logs_client=logs,
    s3_client=s3_client,
    bucket_name=os.getenv("S3_BUCKET")
)
# To stop monitoring, press Esc followed by double-clicking 'i'

<a id="step8"></a>

## Step 8: Connect to the Bedrock Knowledge Base

Documents are parsed and stored in S3. The next step is making them **searchable** by ingesting into the Bedrock Knowledge Base.

The knowledge base was pre-configured to:
- Point to S3 `output/medical_chunks/` folder as the data source
- Use **Amazon Titan** for vector embeddings
- Store vectors in **OpenSearch Serverless** for fast similarity search

In [ ]:
# List all your knowledge bases
bedrock_agent = session.client("bedrock-agent")

print("All Knowledge Bases in your account:")
kb_response = bedrock_agent.list_knowledge_bases()

for kb in kb_response.get("knowledgeBaseSummaries", []):
    print(f"\nKnowledge Base: {kb['name']}")
    print(f"   ID: {kb['knowledgeBaseId']}")
    print(f"   Status: {kb['status']}")
    print(f"   Updated: {kb['updatedAt']}")

    # Get data sources for this knowledge base
    ds_response = bedrock_agent.list_data_sources(
        knowledgeBaseId=kb['knowledgeBaseId']
    )

    for ds in ds_response.get("dataSourceSummaries", []):
        print(f"   Data Source: {ds['name']}")
        print(f"      ID: {ds['dataSourceId']}")
        print(f"      Status: {ds['status']}")

In [ ]:
BEDROCK_KB_ID = <your_bedrock_kb_id>
DATA_SOURCE_ID = <your_data_source_id>

<a id="step9"></a>


## Step 9: Ingest Documents into the Knowledge Base

**Ingestion** syncs parsed documents from S3 into the knowledge base:

1. Knowledge base reads new/modified JSON files from S3
2. Creates vector embeddings for each chunk
3. Stores vectors in the database for fast similarity search

Once complete, you can query with natural language and retrieve relevant document sections.

In [ ]:
response = bedrock_agent.start_ingestion_job(
    knowledgeBaseId=BEDROCK_KB_ID,
    dataSourceId=DATA_SOURCE_ID
)

job_id = response.get("ingestionJob", {}).get("ingestionJobId")
print("✅ Knowledge base sync initiated.")
print(f"   Job ID: {job_id}")

<a id="step10"></a>


## Step 10: Create the Search Tool with Visual Grounding

Create a **search tool** for the agent that adds **visual grounding** - linking extracted information back to exact locations in source documents.

The tool flow:
1. Query Bedrock knowledge base using **hybrid search** (keyword + semantic)
2. For chunk JSON files, parse metadata (chunk_id, page, bbox, type)
3. **Generate cropped chunk images** from source PDFs
4. Upload images to S3 and return **presigned URLs**
5. Format response with source, page, chunk type, and image URL

<div align="left">
    <img src="images/tool.png" width="900">
</div>

In [ ]:
from datetime import datetime
import strands
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager
from visual_grounding_helper import (
    extract_chunk_id_from_markdown,
    extract_chunk_image  # Using extract_chunk_image for cropped images
)

@strands.tool
def search_knowledge_base(query: str) -> str:
    """Search the Bedrock knowledge base for relevant medical documents with visual grounding."""
    try:
        # Ensure we have the required environment variables
        kb_id = os.getenv("BEDROCK_KB_ID")  
        bucket = os.getenv("S3_BUCKET")
        if not kb_id:
            return "Error: Knowledge base ID not configured. Please set BEDROCK_KB_ID environment variable."

        # 1. Query the knowledge base using hybrid search 
        
        # Create runtime client if needed
        bedrock_agent_runtime = session.client("bedrock-agent-runtime")
        
        # Query the Knowledge Base with 5 results as requested
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={"text": query},
            retrievalConfiguration={
                "vectorSearchConfiguration": {
                    "numberOfResults": 5,
                    "overrideSearchType": "HYBRID"  # Use hybrid search for better results
                }
            }
        )
        
        # Get results and sort by score (higher score = more relevant)
        raw_results = response.get("retrievalResults", [])
        sorted_results = sorted(raw_results, key=lambda x: x.get("score", 0), reverse=True)
        
        results = []
        seen_chunk_ids = set()  # Track seen chunk IDs to avoid duplicates

        # 2. For each result, get the location and check if this is a chunk JSON file from medical_chunks folder
        for result in sorted_results:
            content = result.get("content", {}).get("text", "")
            score = result.get("score", 0)
            location = result.get("location", {})
            
            # Get source file from S3 location
            s3_location = location.get("s3Location", {})
            source_uri = s3_location.get("uri", "")
            source_file = source_uri.split("/")[-1] if source_uri else "Unknown source"
            
            # Initialize variables
            chunk_id = None
            visual_info = None
            cropped_image_url = None
            chunk_type = "text"
            page = None
            bbox = None
            source_document = None
            
            # Check if this is a chunk JSON file from medical_chunks folder
            if source_file.endswith('.json') and 'chunks' in source_uri:
                try:
                    # 3. Get chunk data & extract the chunk metadata 
                    # This is a chunk file - parse it directly to get all metadata
                    chunk_key = source_uri.replace(f"s3://{bucket}/", "")
                    chunk_response = s3_client.get_object(Bucket=bucket, Key=chunk_key)
                    chunk_data = json.loads(chunk_response['Body'].read().decode('utf-8'))
                    
                    # Extract all metadata from chunk JSON
                    chunk_id = chunk_data.get('chunk_id', '')
                    chunk_type = chunk_data.get('chunk_type', 'text')
                    page = chunk_data.get('page', 0)
                    bbox = chunk_data.get('bbox', [0, 0, 1, 1])
                    source_document = chunk_data.get('source_document', '')
                    
                    # The text might be in the chunk data or in the content
                    text = chunk_data.get('text', content)
                    
                    # Skip if we've already seen this chunk ID
                    if chunk_id and chunk_id in seen_chunk_ids:
                        continue
                    seen_chunk_ids.add(chunk_id)
                    
                    # 4. Generate cropped chunk image
                    if chunk_id and source_document:
                        source_pdf_key = f"input/medical/{source_document}.pdf"
                        try:
                            s3_client.head_object(Bucket=bucket, Key=source_pdf_key)
                            cropped_image_url = extract_chunk_image(
                                s3_client=s3_client,
                                bucket=bucket,
                                source_pdf_key=source_pdf_key,
                                bbox=bbox,
                                page_num=page,
                                chunk_id=chunk_id,
                                source_document=source_document,
                                highlight=True,
                                padding=10
                            )
                        except:
                            pass  # PDF not found
                            
                except Exception as e:
                    # Fallback if can't parse chunk file
                    pass
            else:
                # Not a chunk file, try to extract chunk ID from markdown
                chunk_id = extract_chunk_id_from_markdown(content)
                
                # Skip if we've already seen this chunk ID
                if chunk_id and chunk_id in seen_chunk_ids:
                    continue
                if chunk_id:
                    seen_chunk_ids.add(chunk_id)
            
            # 5. Format result with all available information
            if cropped_image_url and chunk_id and page is not None:
                # Complete visual grounding available
                result_text = f"""
                **Source:** {source_document or source_file} (Relevance: {score:.2f})
                📄 **Chunk ID:** {chunk_id}
                📍 **Page:** {page}
                🏷️ **Chunk Type:** {chunk_type}
                🔍 **Cropped Chunk Image:** {cropped_image_url}
                
                **Content:**
                {content}"""
                results.append(result_text)
            elif chunk_id and page is not None:
                # Partial visual info (no image but has metadata)
                result_text = f"""
                **Source:** {source_document or source_file} (Relevance: {score:.2f})
                📄 **Chunk ID:** {chunk_id}
                📍 **Page:** {page}
                🏷️ **Chunk Type:** {chunk_type}
                📦 **Bbox:** {bbox if bbox else 'Not available'}
                
                **Content:**
                {content}"""
                results.append(result_text)
            else:
                # No visual grounding available - use content hash as unique ID
                content_hash = hash(content[:200])  # Hash first 200 chars for uniqueness
                if content_hash in seen_chunk_ids:
                    continue
                seen_chunk_ids.add(content_hash)
                
                clean_source = source_file.replace('_grounding.json', '').replace('.json', '').replace('.md', '')
                result_text = f"""**Source:** {clean_source} (Relevance: {score:.2f})
                                **Content:**{content}"""
                results.append(result_text)
        
        if results:
            # Return only top 5 most relevant results with visual references
            return "\n\n---\n\n".join(results[:5])
        else:
            return f"No documents found for query: '{query}'. The knowledge base may be empty or still processing."
            
    except Exception as e:
        error_msg = str(e)
        if "ResourceNotFoundException" in error_msg:
            return f"Error: Knowledge base {kb_id} not found. Please verify the BEDROCK_KB_ID is correct."
        elif "ValidationException" in error_msg:
            return f"Error: Invalid query or configuration. Details: {error_msg}"
        else:
            return f"Error searching knowledge base: {error_msg}"

### Testing the Search Function

Verify the search tool works before creating the agent:

In [ ]:
# Test the search function before creating agent
print("Testing knowledge base search function...")
test_result = search_knowledge_base("common cold symptoms")
print(f"Test result: {test_result[:200]}...")

if "Error" in test_result:
    print("\n Knowledge base search is not working. Checking configuration...")
    print(f"Current KB ID: {os.getenv('BEDROCK_KB_ID')}")
    print(f"Current Region: {os.getenv('AWS_REGION')}")
else:
    print("\n✅ Knowledge base search is working!")

In [ ]:
print(test_result)

<a id="step11"></a>


## Step 11: Create Memory for the Agent

AWS Bedrock AgentCore provides **persistent memory** so your agent remembers conversations and learns preferences over time.

### Memory Strategies

| Strategy | What It Does | Example |
|----------|--------------|--------|
| **Summary** | Summarizes past sessions | "Last time we discussed cold treatments" |
| **User Preference** | Learns user preferences | "User prefers short answers" |
| **Semantic** | Extracts and stores facts | "User mentioned they have allergies" |

Memory persists across sessions, enabling personalized responses.

In [ ]:
# Initialize memory client
memory_client = MemoryClient(region_name=os.getenv("AWS_REGION", "us-west-2"))

# Try to list existing memories first
try:
    existing_memories = memory_client.gmcp_client.list_memories()
    memory_list = existing_memories.get('memories', [])
    
    # Get all MedicalAgentMemory instances and use the most recent
    medical_memories = [m for m in memory_list if 'MedicalAgentMemory' in m.get('id', '')]
    
    if medical_memories:
        # Sort by creation date and take the most recent
        medical_memories.sort(key=lambda x: x.get('createdAt', ''), reverse=True)
        existing_medical_memory = medical_memories[0]
        MEMORY_ID = existing_medical_memory.get('id')
        print(f"Found {len(medical_memories)} existing MedicalAgentMemory instance(s)")
        print(f" Using most recent memory: {MEMORY_ID}")
        print(f"   Created: {existing_medical_memory.get('createdAt', 'N/A')}")
        print(f"   Status: {existing_medical_memory.get('status', 'N/A')}")
    else:
        # Only create if none exist
        raise Exception("No existing MedicalAgentMemory found, will create new one")
        
except Exception as e:
    # Create new memory only if none exists
    print(f" Creating new memory... (Reason: {e})")
    try:
        # Add timestamp to make name unique
        comprehensive_memory = memory_client.create_memory_and_wait(
            name=f"MedicalAgentMemory_{datetime.now().strftime('%Y%m%d_%H%M%S')}", 
            description="Memory for medical document analysis with user preferences",
            strategies=[
                {
                    "summaryMemoryStrategy": {
                        "name": "SessionSummarizer",
                        "namespaces": ["/summaries/{actorId}/{sessionId}"]
                    }
                },
                {
                    "userPreferenceMemoryStrategy": {
                        "name": "PreferenceLearner",
                        "namespaces": ["/preferences/{actorId}"]
                    }
                },
                {
                    "semanticMemoryStrategy": {
                        "name": "FactExtractor",
                        "namespaces": ["/facts/{actorId}"]
                    }
                }
            ]
        )
        MEMORY_ID = comprehensive_memory.get('id')
        print(f" New memory created: {MEMORY_ID}")
    except Exception as create_error:
        print(f" Could not create memory: {create_error}")
        print("Continuing without memory functionality...")
        MEMORY_ID = None

### Configuring the Memory Session

The session manager requires two identifiers:
- **Actor ID**: Who is using the agent (enables personalization)
- **Session ID**: Unique identifier for this conversation

In [ ]:
# Set up memory configuration if memory exists
if MEMORY_ID:
    ACTOR_ID = f"medical_user_{datetime.now().strftime('%H%M%S')}"
    SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    print(f"   Actor: {ACTOR_ID}")
    print(f"   Session: {SESSION_ID}")

    # Configure memory
    memory_config = AgentCoreMemoryConfig(
        memory_id=MEMORY_ID,
        session_id=SESSION_ID,
        actor_id=ACTOR_ID
    )

    # Create session manager
    session_manager = AgentCoreMemorySessionManager(
        agentcore_memory_config=memory_config,
        region_name=os.getenv("AWS_REGION", "us-west-2")
    )
else:
    session_manager = None
    print("Agent will run without memory")

<a id="step12"></a>


## Step 12: Create the Strands Agent

Bring everything together into a **Strands Agent** configured with:
- **Model**: Claude via Bedrock as the underlying LLM
- **System prompt**: Instructions defining agent behavior
- **Session manager**: Memory for preferences and history
- **Tools**: The `search_knowledge_base` function with visual grounding

In [ ]:
from strands import Agent

# Create the agent with memory and tools
medical_agent = Agent(
    model=os.getenv("BEDROCK_MODEL_ID"),
    name="Medical Document Analyzer with Memory",
    description="Expert agent for medical documents with conversation memory",
    system_prompt="""
        You are a medical document analysis assistant with memory capabilities and visual grounding support.
        You remember our conversations, user preferences, and important facts.
        
        Your capabilities:
        - Search and analyze medical documents from the knowledge base
        - Provide visual grounding information showing exact locations in documents
        - Display page numbers and bounding box coordinates when available
        - Reference annotated images that highlight specific document regions
        - Remember user preferences and conversation history
        - Provide personalized, contextual responses
        - Learn from interactions to improve future responses
        
        IMPORTANT: When you receive search results that include visual grounding information, you MUST include:
        - Page numbers where information was found
        - Location coordinates showing exact position on the page
        - Annotated image URLs that show highlighted text regions
        
        When search results contain these visual markers, preserve them in your response. Do not summarize away the visual grounding details.
        
        Visual grounding format to preserve:
        - **Page:** [number] - shows which page contains the information
        - **Location:** [coordinates] - shows exact position on the page
        - **Annotated Image:** [URL] - provides visual highlight of the referenced text
        
        You have access to medical documents about common cold, treatments, and symptoms.
        Always provide evidence-based insights from the documents with visual references when available.
        When visual grounding is provided in search results, include it in your response to help users see exactly where information comes from.
        """,
    session_manager=session_manager,
    tools=[search_knowledge_base]
)

print(f"\n✅ Medical agent ready with memory and visual grounding!")
print(f"   Model: {os.getenv('BEDROCK_MODEL_ID')}")
print(f"   Tools: {medical_agent.tool_names}")
print("\nThe agent will now:")
print("   - Remember your preferences and conversation history")
print("   - Show exact locations in documents when available")
print("   - Provide visual grounding with page numbers and coordinates")

<a id="step13"></a>


## Step 13: Interactive Chat

Your medical document agent is ready! Start an interactive chat session to:
- Ask questions about medical documents
- See visual grounding with page numbers and image URLs
- Tell the agent your preferences (e.g., "I prefer short answers")
- Watch the agent remember preferences in future sessions

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
print("=" * 70)
print("Medical Agent - Interactive Chat with Visual Grounding")
print("=" * 70)
print("\nAsk questions about medicine.")
print("Type 'exit', 'quit', or 'bye' to end the conversation.")
print("=" * 70 + "\n")

conversation_num = 0

while True:
    try:
        user_input = input("\nYou: ").strip()

        if not user_input:
            continue

        if user_input.lower() in ['exit', 'quit', 'bye', 'q']:
            print("\n Ending conversation. Goodbye!")
            break

        conversation_num += 1

        # Display the question prominently
        print("\n" + "─" * 70)
        print(f"Question #{conversation_num} [{datetime.now().strftime('%H:%M:%S')}]")
        print(f"   \"{user_input}\"")
        print("─" * 70)

        print("\nAgent Response:")
        print("   Processing...\n")

        # Get and display the response
        result = medical_agent(user_input)
        print(result)

        print("\n" + "=" * 70)

    except KeyboardInterrupt:
        print("\n\n Conversation interrupted. Goodbye!")
        break
    except Exception as e:
        print(f"\n Error: {e}")
        print("Please try again or type 'exit' to quit.")

## Summary

Here's what you built in Lab 6:

| Component | Service | Function |
|-----------|------------|--------|
| **Storage** | Amazon S3 | Store raw PDFs and parsed outputs |
| **Trigger** | AWS Lambda | Serverless document parsing with ADE |
| **Vector DB** | Bedrock Knowledge Base | Semantic search over documents |
| **Agent** | Strands Agents | Conversational interface |
| **Memory** | AgentCore Memory | Remember preferences and history |

You can extend this pipeline to handle other document types, add more tools, or integrate with other AWS services as needed.